In [ ]:
"""
Created on Thu Oct 3 11:19:21 2021
@author: cliu
"""
import os
import yaml
import pandas as pd
import numpy as np
from datetime import date
from os import listdir
from os.path import isfile

_join = os.path.join
toolpath = os.getcwd()

CONFIG = r'T:/ABM/ABM_FY22/RSM/VisionEval/Model/Update_Automations/settings.yml'
with open(CONFIG) as cff:
    config =yaml.safe_load(cff)
target_path = ''.join(config['target_path'])
scenpath = ''.join(config['scenpath'])
scenario_list = config['scenario_list']
azone_electricity_carbon_intensity = config['azone_electricity_carbon_intensity']
Marea = config['Marea']
AzoneCount = config['AzoneCount']
VehOwnFlatRateFee = config['VehOwnFlatRateFee.2015']
VehOwnAdValoremTax = config['VehOwnAdValoremTax']
PaydHhProp = config['PaydHhProp']
AveComSvcVehicleAge = config['AveComSvcVehicleAge']
ComSvcPow = config['ComSvcPow']
hvytrk_powertrain_prop = config['hvytrk_powertrain_prop']
PropChargingAvail = config['PropChargingAvail']

df_geo = pd.read_csv(_join(target_path, 'defs\geo.csv'))
df_geo = df_geo[['Azone', 'Bzone']]
df_geo.rename(columns={'Bzone':'taz'}, inplace=True) 

In [ ]:
# Initialize-vepowertrainsandfuels/6.1 CalculateCarbonIntensity - azone_electricity_carbon_intensity.csv¶

# azone_electricity_carbon_intensity.csv
df_carbon = pd.read_csv(_join(target_path, r'inputs\azone_electricity_carbon_intensity.csv'))
for i in range(len(azone_electricity_carbon_intensity)):
    df_carbon.loc[df_carbon['Year'] == scenario_list[i][0] , 'ElectricityCI'] = int(azone_electricity_carbon_intensity[i])
df_carbon['ElectricityCI'] = df_carbon['ElectricityCI'].astype('int64')
df_carbon.to_csv(_join(target_path, r'inputs\azone_electricity_carbon_intensity.csv'), index = False) 

In [ ]:
6.2 Assign Household Vehicle Powertrains - region_carsvc_powertrain_prop.csv

In [ ]:
# region_carsvc_powertrain_prop.csv
df_ori = pd.read_csv(_join(toolpath, r'region_carsvc_powertrain_prop.csv'))
i = 0
for year, scenario in scenario_list:
    df_ori['Year'][i] = year
    i += 1
df_ori.to_csv(_join(target_path, r'inputs\region_carsvc_powertrain_prop.csv'), index = False) 

In [ ]:
# azone_charging_availability.csv
base_year = scenario_list[0][0]
i = 0
for year, scenario in scenario_list:
    df_mgra = pd.read_csv(_join(scenpath, scenario, 'input\mgra13_based_input' + str(year) + '.csv'))
    df_mgra = df_mgra[['taz','hh_sf','hh_mf','refueling_stations']]
    df_mgra = df_mgra.merge(df_geo, on=['taz'], how = 'inner') 
    df_AzoneGro = df_mgra.groupby(['Azone'])['hh_sf', 'hh_mf', 'refueling_stations'].sum().reset_index()
    max_value = df_AzoneGro['refueling_stations'].max()
    
    df_AzoneGro['PropSFChargingAvail'] = PropChargingAvail [i][0]
    df_AzoneGro['PropMFChargingAvail'] = PropChargingAvail [i][1]   
    df_AzoneGro['PropGQChargingAvail'] = PropChargingAvail [i][2]
    df_AzoneGro.insert(loc=1, column='Year', value = year)
    df_AzoneGro.rename(columns={'Azone':'Geo'}, inplace=True)  
    df_AzoneGro=df_AzoneGro.drop(['hh_sf', 'hh_mf', 'refueling_stations'], axis=1)    
    
    if year == base_year:
            df_AzoneGro.to_csv(_join(target_path,r'inputs\azone_charging_availability.csv'), mode='w', header = True, index = False)
    else:
        df_AzoneGro.to_csv(_join(target_path,r'inputs\azone_charging_availability.csv'), mode='a', header = False, index = False)
    i += 1 